In [1]:

import argparse
import numpy as np
from data_loader import load_data
import logging
np.random.seed(2022)
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
args = argparse.Namespace()
args.dataset="naver-toy"
args.dim=32
args.n_hop=2
args.kge_weight=0.01
args.l2_weight=1e-7
args.lr=0.02
args.batch_size=256
args.n_epoch=1
args.n_memory=32
args.item_update_mode="plus_transform"
args.using_all_hops=True
args.use_cuda = True
args.show_topk =True
args.Ks=[20,40,60,80,100]
args.test_flag = "part"

import numpy as np
import torch
import logging
from model import RippleNet
from sklearn.metrics import roc_auc_score, f1_score
from train import train, _get_user_record, get_feed_dict
batch_size = args.batch_size


import multiprocessing
Ks = args.Ks
BATCH_SIZE = args.batch_size
batch_test_flag = True
cores = multiprocessing.cpu_count() // 2
device = torch.device("cuda")

In [3]:
data_info = load_data(args)

reading rating file ...
splitting dataset ...
reading KG file ...
constructing knowledge graph ...
constructing ripple set ...


In [4]:
show_loss = False

# Train Function

In [5]:
train_data = data_info[0]
eval_data = data_info[1]
test_data = data_info[2]
n_entity = data_info[3]
n_relation = data_info[4]
ripple_set = data_info[5]

model = RippleNet(args, n_entity, n_relation)
if args.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    args.lr,
)


In [6]:
n_entity

7348

In [7]:

    # training
np.random.shuffle(train_data)

start = 0
return_dict = model(*get_feed_dict(args, model, train_data, ripple_set, start, start + args.batch_size))
loss = return_dict["loss"]

optimizer.zero_grad()
loss.backward()
optimizer.step()

# start += args.batch_size
# if show_loss:
#     print('%.1f%% %.4f' % (start / train_data.shape[0] * 100, loss.item()))


In [8]:
start =0
items, labels, memories_h, memories_r, memories_t = get_feed_dict(args, model, train_data, ripple_set, start, start + args.batch_size)

In [9]:
memories_t[0].shape # (batch_size, dim)

torch.Size([256, 32])

In [10]:
train_user_set = _get_user_record(train_data,True)

In [11]:
test_user_set = _get_user_record(test_data, True)

In [12]:
user_dict = {
    "train_user_set" : train_user_set,
    "test_user_set" : test_user_set
}

In [13]:
user_dict["test_user_set"]

{8169: {392, 1663, 2191},
 5921: {246, 1161},
 737: {24, 160, 4309},
 4989: {2651},
 11429: {3105, 3109, 4372, 5859},
 770: {25, 1326, 1412},
 3972: {812, 1458, 2180, 4037, 4198, 4214, 4335, 4804, 5343},
 8456: {1120, 5167},
 11370: {1286, 1408, 2806, 3192, 3788, 6263, 6625},
 1174: {28, 3342, 7121},
 12693: {2327},
 12085: {3614, 4340, 5393, 6053},
 12861: {2462, 2582, 3761, 4808, 5317, 5319},
 8732: {291, 298, 595, 2218, 5082, 5948, 7025},
 3975: {5155},
 5664: {4453, 4957},
 8755: {1333, 2151, 3639},
 2101: {120, 168, 1310, 2144},
 2350: {184, 1398, 2190},
 8233: {1804, 2198, 5243, 6190, 6768},
 512: {232, 703, 1075, 1677, 4688},
 48: {2,
  42,
  44,
  58,
  59,
  62,
  64,
  65,
  67,
  80,
  88,
  91,
  123,
  130,
  134,
  139,
  143,
  161,
  164,
  173,
  184,
  201,
  203,
  232,
  246,
  258,
  266,
  294,
  312,
  344,
  348,
  351,
  362,
  364,
  369,
  386,
  391,
  407,
  413,
  415,
  417,
  421,
  430,
  440,
  454,
  463,
  467,
  501,
  523,
  534,
  544,
  548,
  57

In [14]:
len(train_user_set.keys())

12855

In [15]:
len(test_user_set.keys())

11032

In [16]:
all_data = np.vstack([train_data, test_data,eval_data])


In [17]:
def _get_item_record(data):
    item_dict = dict()
    for rating in data:
        user = rating[0]
        item = rating[1]
        label = rating[2]

        if label == 1:
            if item not in item_dict:
                item_dict[item] = set()
            item_dict[item].add(user)

    return item_dict

In [18]:
all_item_set = _get_item_record(all_data)
all_user_set = _get_user_record(all_data, True)


In [19]:
len(all_item_set.keys())

7237

In [20]:
n_items = len(all_user_set.keys())
n_users = len(all_item_set.keys())
n_nodes = n_entity + n_users

In [21]:
n_params ={
    "n_items" : n_items,
    "n_users" : n_users,
    "n_entities" : n_entity,
    "n_relations" : n_relation,
    "n_nodes" : n_nodes
}

In [22]:
len(model.o_list)

2

In [59]:
from metrics import *
# from .parser import parse_args
import multiprocessing
import heapq
from time import time

def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

def get_auc(item_score, user_pos_test):
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]
    posterior = [x[1] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = AUC(ground_truth=r, prediction=posterior)
    return auc

def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc

def get_performance(user_pos_test, r, auc, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(precision_at_k(r, K))
        recall.append(recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(ndcg_at_k(r, K, user_pos_test))
        hit_ratio.append(hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio), 'auc': auc}


def test_one_user(x):
    test_flag = args.test_flag
    # user u's ratings for user u
    rating = x[0] # rate_batch
    
    # uid
    u = x[1]
    # user u's items in the training set
    try:
        training_items = train_user_set[u]
    except Exception:
        training_items = []
    # user u's items in the test set
    user_pos_test = test_user_set[u]
    # print(user_pos_test)
    all_items = set(range(0, n_entity))

    test_items = list(all_items - set(training_items))
    # print(len(test_items))
    if test_flag == 'part':
        r, auc = ranklist_by_heapq(user_pos_test, test_items, rating, Ks)
    else:
        r, auc = ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

    return get_performance(user_pos_test, r, auc, Ks)

# def test(model, user_dict, n_params):
#     result = {'precision': np.zeros(len(Ks)),
#               'recall': np.zeros(len(Ks)),
#               'ndcg': np.zeros(len(Ks)),
#               'hit_ratio': np.zeros(len(Ks)),
#               'auc': 0.}

#     global n_users, n_items
#     n_items = n_params['n_items']
#     n_users = n_params['n_users']

#     global train_user_set, test_user_set
#     train_user_set = user_dict['train_user_set']
#     test_user_set = user_dict['test_user_set']

#     pool = multiprocessing.Pool(cores)

#     u_batch_size = BATCH_SIZE
#     i_batch_size = BATCH_SIZE

#     test_users = list(test_user_set.keys())
#     n_test_users = len(test_users)
#     n_user_batchs = n_test_users // u_batch_size + 1

#     count = 0

    
    
#     for u_batch_id in range(n_user_batchs):
#         start = u_batch_id * u_batch_size
#         end = (u_batch_id + 1) * u_batch_size

#         user_list_batch = test_users[start: end]
#         user_batch = torch.LongTensor(np.array(user_list_batch)).to(device)
#         u_g_embeddings = model.get_user_embeddings()

#         if batch_test_flag:
#             # batch-item test
#             n_item_batchs = n_items // i_batch_size + 1
#             rate_batch = np.zeros(shape=(len(user_batch), n_items))

#             i_count = 0
#             for i_batch_id in range(n_item_batchs):
#                 i_start = i_batch_id * i_batch_size
#                 i_end = min((i_batch_id + 1) * i_batch_size, n_items)

#                 item_batch = torch.LongTensor(np.array(range(i_start, i_end))).view(i_end-i_start).to(device)
#                 i_g_embddings = model.entity_emb(item_batch)

#                 i_rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()

#                 rate_batch[:, i_start: i_end] = i_rate_batch
#                 i_count += i_rate_batch.shape[1]

#             assert i_count == n_items
#         else:
#             # all-item test
#             item_batch = torch.LongTensor(np.array(range(0, n_items))).view(n_items, -1).to(device)
#             # i_g_embddings = entity_gcn_emb[item_batch]
#             # rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()
#             rate_batch = model.rating(u_g_embeddings, i_g_embddings).detach().cpu()

#         user_batch_rating_uid = zip(rate_batch, user_list_batch)
#         batch_result = pool.map(test_one_user, user_batch_rating_uid)
#         count += len(batch_result)

#         for re in batch_result:
#             result['precision'] += re['precision']/n_test_users
#             result['recall'] += re['recall']/n_test_users
#             result['ndcg'] += re['ndcg']/n_test_users
#             result['hit_ratio'] += re['hit_ratio']/n_test_users
#             result['auc'] += re['auc']/n_test_users

#     assert count == n_test_users
#     pool.close()
#     return result


In [55]:
len(test_user_set[8169])

3

In [64]:
n_params

{'n_items': 12855,
 'n_users': 7237,
 'n_entities': 7348,
 'n_relations': 3,
 'n_nodes': 14585}

In [34]:
result = {'precision': np.zeros(len(Ks)),
              'recall': np.zeros(len(Ks)),
              'ndcg': np.zeros(len(Ks)),
              'hit_ratio': np.zeros(len(Ks)),
              'auc': 0.}

In [35]:
n_items = n_params["n_items"]
n_users = n_params["n_users"]

train_user_set = user_dict["train_user_set"]
test_user_set = user_dict["test_user_set"]

In [36]:
pool = multiprocessing.Pool(cores)
u_batch_size = BATCH_SIZE
i_batch_size = BATCH_SIZE

test_users = list(test_user_set.keys())
n_test_users = len(test_users)
n_user_batchs = n_test_users // u_batch_size +1


11032
11264
[ 6.6445117  -7.77216864  0.15359032 ... -3.93493724 -1.51191223
  6.15885687][-5.12441874 -0.07825124 -0.49265409 ... 10.7420187  -5.36049175
 -8.75077724][ -6.36109066  -5.87269545   9.64050579 ...  -0.60761386 -22.65194702
  -1.89841747]


[-2.39715624  8.07489395  0.79500306 ... 12.97413063 10.94895935
 -7.09870148][-13.25249672  -3.72790384   3.61006522 ...   6.0908947    3.93200397
  -2.79388022]

[ 5.29571819 12.15299892 12.8188076  ... -2.017138   -5.1346488
 -2.28922725]
[ -5.28910494  -3.86704636  -0.90452623 ... -16.90884018   4.58296108
   1.39994514][ 1.00096970e+01 -1.46636963e-02  2.41550446e-01 ...  2.24593906e+01
  1.07175839e+00 -4.86158419e+00][-2.71198797 15.75195503  1.94015002 ... 28.10194969 -7.74161386
 -1.98919034]


[ 0.11363745 19.67926407  3.35412574 ...  9.59677696 -6.65138531
  3.91620255]
[ 3.6522758  -0.96816421  3.58457875 ... 12.98495293 -5.72175741
 -0.66753411]
[-3.63229108 -3.10968232 -5.08478308 ... -6.11350679  4.46241808
  1.04911089]

In [37]:
11032 - (11032 // batch_size) * batch_size


24

In [63]:
u_count = 0
for u_batch_id in range(n_user_batchs):
    start = u_batch_id * u_batch_size
    end = min((u_batch_id+1)*u_batch_size, n_test_users)

    user_list_batch = test_users[start:end]
    user_batch = torch.LongTensor(np.array(user_list_batch)).to(device)
    user_emb = model.get_user_embeddings(*get_feed_dict(args, model, test_data, ripple_set, start, end))
    
    n_items_batch = n_entity // i_batch_size + 1
    rate_batch = np.zeros(shape=(len(user_batch), n_entity))

    i_count = 0

    for i_batch_id in range(n_items_batch):
        i_start = i_batch_id * i_batch_size
        i_end = min((i_batch_id+1)*i_batch_size, n_entity)

        item_batch = torch.LongTensor(np.array(range(i_start, i_end))).view(i_end-i_start).to(device)
        item_emb = model.entity_emb(item_batch)

        i_rate_batch = model.rating(user_emb, item_emb).detach().cpu()
        rate_batch[:, i_start:i_end] = i_rate_batch
    # print(rate_batch.shape, len(user_list_batch))
    user_batch_rating_uid = zip(rate_batch, user_list_batch)
    # batch_result = list()
    for rating,uid in user_batch_rating_uid:
        x = (rating, uid)
        # print(test_one_user(x))
        re = test_one_user(x)
        print(re)
        # result["precision"] = re["precision"] / n_test_users
        # result["recall"] = re["recall"] / n_test_users
        # result["ndcg"] = re["ndcg"] / n_test_users
        # result["hit_ratio"] = re["hit_ratio"] / n_test_users
        # result["auc"] = re["auc"] / n_test_users


{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': array([0., 0., 0., 0., 0.]), 'auc': 0.0}
{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': array([0., 0., 0., 0., 0.]), 'auc': 0.0}
{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': array([0., 0., 0., 0., 0.]), 'auc': 0.0}
{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': array([0., 0., 0., 0., 0.]), 'auc': 0.0}
{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': array([0., 0., 0., 0., 0.]), 'auc': 0.0}
{'recall': array([0., 0., 0., 0., 0.]), 'precision': array([0., 0., 0., 0., 0.]), 'ndcg': array([0., 0., 0., 0., 0.]), 'hit_ratio': arr

In [62]:
result

{'precision': array([0., 0., 0., 0., 0.]),
 'recall': array([0., 0., 0., 0., 0.]),
 'ndcg': array([0., 0., 0., 0., 0.]),
 'hit_ratio': array([0., 0., 0., 0., 0.]),
 'auc': 0.0}

In [ ]:
n_user_batchs * 256 + 24